# Documentation  

Anaconda 2019.07 버전의 python 3.7.4 defalut 값으로 진행  
  
Python = 3.7.4  
Pandas = 0.25.1   
Numpy = 1.16.5  
urllib3 = 1.24.2  
BeautifulSoup = 4.8.0  
requests = 2.22.0   
selenium = 3.141.0  

In [1]:
#import module

#python defalut module
import time
import re
import os
from datetime import datetime,timedelta
from IPython.display import Image

import pandas as pd
import numpy as np

import urllib3
from bs4 import BeautifulSoup
import requests

import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 이전 문서와 중복되면 해당 기사의 row index를 기록하는 코드
def duplicate_title(prev_list,prev_title,save_list) :
    for row, title in enumerate(prev_title) :
        if title in prev_list :
            save_list.append(row) 

def common_tag_del(content) :
    for target in content.find_all('script'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()
        
    for target in content.find_all('iframe'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()
            
        
def df_common_prep(dataframe) :
    dataframe['title'] = dataframe.title.str.replace('\\',"") # 따옴표가  \' 형태로 기록되어 해당 \를 공란으로 대체
    dataframe = dataframe[~dataframe['title'].str.contains('MOU',na=False)] # 유용하지 않은 기사인 MOU 제거
    dataframe.drop_duplicates(subset=['title'], keep='last',inplace=True) # 기사 중 마지막 기사만 남기고 중복 제거
    dataframe.dropna().reset_index(drop=True, inplace=True) # 데이터 편집을 위한 NA제거 및 index reset
    dataframe.head()
    

def itworld_cio_date(soup, date_class) :
    date = soup.find_all("div",{"class":date_class})
    # 날짜가 기록된 태그 저장
    start_date = datetime.now() # 현재 날짜 저장

    for j in date :
        date_list = j.get_text() #날짜 추출

        if '시간' in date_list : # 날짜가 n시간 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', date_list) 
            final_date = str(datetime.now() - timedelta(hours=int(date_find[0])))
            total_date.append(final_date)

        elif '분' in date_list : # 날짜가 n분 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', date_list)
            final_date = str(datetime.now() - timedelta(minutes=int(date_find[0])))
            total_date.append(final_date)

        elif '일' in date_list: # 날짜가 n일 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', date_list)
            final_date = str(datetime.now() - timedelta(days=int(date_find[0])))
            total_date.append(final_date)    

        elif bool(re.search(r'\d{3,4}.\d{1,2}.\d{2}',date_list)) == True : # yyyy.mm.dd로 표기될 경우 처리
            final_date = date_list.replace('.',"-")
            total_date.append(final_date)
            


    
def article_post(news_title,news_text) :
    
    icu_title = driver.find_element_by_id("Title_fa564e0f-0c70-4ab9-b863-0177e6ddd247_$TextField") # 제목란 클릭

    icu_title.send_keys(news_title) # 작성 제목에 기사 제목 입력
    time.sleep(5)
    
    # iframe 전환

    # 작성 게시글 내부의 inline frame(iframe) 2개의 web page가 삽입되어 있습니다.
    # 기사를 입력하기 위해서는 내부에 들어가서 작성해야 합니다.
    # 이를 위해 tag_name으로 iframe이 입력된 전체 리스트를 찾고 
    # 기사 제목을 입력할 수 있는 iframe[0] 내부의 두번째 iframe에 접속하여 기사 본문을 게시하는 코드를 작성하였습니다.
    
    iframes = driver.find_elements_by_tag_name('iframe') # 모든 iframe tage_name을 검색
    driver.switch_to.frame(iframes[0]) # 그 중 본문을 작성할 수 있는 첫번째 iframe 선택
    time.sleep(2)

    second_iframe = driver.find_elements_by_tag_name('iframe') # 본문에 접속하기 위해서는 iframe[0]내부의 iframe에 접속하여 작성해야됨
    driver.find_element_by_xpath('//*[@id="ue_dext5editorsource"]/a').click() # I.C.U 본문작성 하위 태그의 HTML태그 클릭 
    # 이를 위해 2번째 iframe을 검색
    driver.switch_to.frame(second_iframe[0])  # 검색한 두 번째 iframe 접속
    icu_news = driver.find_element_by_id("dext_body") # 해당 iframe 내의 기사를 입력할 수 있는 id = dext_body 검색
    icu_news.clear()
    icu_news.send_keys(news_text) # 해당 부분에 기사 내용 입력
    time.sleep(5)
    
    driver.switch_to_default_content()# 해당 iframe에 벗어나 상위 웹페이지로 연결


### 다른 사이트의 경우 IT 이외의 과학 뉴스 등 다른 분야의 기사가 많이 혼재하여
> 네이버 뉴스 IT/과학 헤드라인  
>    
> ITWorld korea의 머신러닝, 클라우드, 빅데이터 파트  
>    
> ZDnet의 인공지능 파트
>
> CIO Korea의 빅데이터, 클라우드, AI파트에서 데이터를 추출하였습니다.

<p> <img src="img/그림2.png" height="300px" width="700px"></p>

#### - 네이버 뉴스의 경우 "헤드라인 더보기" 전까지 기사의 제목과 URL을 추출후 DataFrame안에 저장하여 기록하고  
  
#### - 이후 저장한 URL에 개별 접속하여 기사의 본문을 가져와서 저장하는 형식으로 진행하였습니다

In [5]:
news = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105' ## IT/과학 헤드라인 뉴스  url

res = requests.get(news, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
# requests이용하여 뉴스 주소 Load, 네이버 크롬 차단 방지 위해 chrome version을 입력하고 뉴스 주소를 설정


# SSL ERROR 방지를 위해 Verify False 설정
soup = BeautifulSoup(res.text, "html.parser") # html기준으로 data parsing

naver = soup.select("#main_content a")  # 기사 제목과 URL 모두 html main_content a에 저장되어 있음. 해당 부분을 select하여 추출


title_list =[] # 제목을 저장하기 위한 list 생성
url_list = [] # URL을 저장하기 위한 list 생성
date_list = [] # 날짜를 저장하기 위한 list 생성

# 제목, 링크, date 출력 ---------------------------
for result in naver:
    if result.string == None: # 값이 존재하지 않을 경우 해당 루프 건너 뜀
        continue
    
    if result.attrs['href'] in ['#'] : # 헤드라인 더보기의 end code가 #이므로 #일 경우 loop문을 Break하여 종료함
        break
    
    if result.attrs['href'].startswith('http') == False : 
        # 해당 href에서 url 코드가 http로 시작하지 않을경우 url이 아니라고 판단하여 해당 부분 skip
        continue
    

    title_list.append(str.strip(result.string)) # 크롤링한 기사 제목을 리스트에 추가
    url_list.append(result.attrs['href']) # 크롤링한 기사 URL을 리스트에 추가
    date_list.append(datetime.today().strftime("%Y-%m-%d")) # 현재 날짜를 리스트에 추가


df = pd.DataFrame({'title': title_list,'url':url_list, 'date':date_list}) # 기사제목, URL, 날짜를 dataframe에 저장
df['title'] = df.title.str.replace('\\',"") # 작은 따옴표가  \' 형태로 기록되어 \를 공란으로 대체
df.head()

,title,url,date
0,"애플, 14년 만에 자체 칩 탑재 '맥북' 출시하나…10일 공개",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
1,"애플 '원모어띵'은 새로운 맥북, 10일 발표",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
2,"“한 가지 소식이 더” 애플, 10일 자체 칩 탑재 맥북 공개할 듯",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
3,"애플, 오는 10일 특별이벤트 실시…""자체개발 칩 탑재한 맥 공개하나""",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
4,“1004 주인 찾습니다!” 400대1 골드번호 도전! [IT선빵!],https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03


<p> <img src="img/그림1.png" height="300px" width="700px"></p>

### 위의 그림처럼 네이버는 기사들이 중복되어 나타나기 때문에 이를  제거하는 작업이 필요합니다
### 중복을 없애기 위해 기사 제목을 통해 중복 뉴스를 찾아 제거하였습니다

In [6]:
# 중복된 기사 list 추출
raw_df = df.copy()
duplicate_list = []
dup_title = soup.find_all("li",{"class":"cluster_item as_line"})
for dup in dup_title:
    text = dup.a.get_text()
    duplicate_list.append(text.replace('\\',""))
duplicate_list

["애플 '원모어띵'은 새로운 맥북, 10일 발표",
 '“한 가지 소식이 더” 애플, 10일 자체 칩 탑재 맥북 공개할 듯',
 '애플, 오는 10일 특별이벤트 실시…"자체개발 칩 탑재한 맥 공개하나"',
 '‘용달블루폰?’ ‘쌈무그린폰?’… 어떤 휴대폰 애칭? [IT선빵!]',
 '"아이폰12 완전체? 20만원 더!"…비싼 애플 악세서리 품질은 \'망작\'? [IT선빵!]',
 '‘209만원→90만원’ 귀족폰 ‘갤폴드’ 15일(?)까지 반값에 판다! [IT선빵!]',
 '[주간 클라우드 동향/10월④] 클라우드 업계, "대형 레퍼런스 잡아라"',
 "[주간 클라우드 동향/10월③] 네이버 B2B도 '네이버 클라우드'로 모은다",
 '[주간 클라우드 동향/10월②] 변신의 귀재 IBM, 이번엔 성공할까']

#### 위에 기사 제목과 비교하면 중복된 기사인 1,2,3 번의 뉴스가 제거된 것을 볼 수 있습니다

또한 main topic인 기사의 경우 중복제거를 위해    
동일한 위치에 기록되는 것을 바탕으로 중복 제거를 시행하였습니다.

In [7]:
# 중복된 기사 제거
title_row = []
duplicate_title(duplicate_list, df.title,title_row)


if len(df) != 0 :
    df.drop([5,7,9,11],inplace=True)
    df.drop(title_row,inplace=True)
    df.reset_index(drop=True,inplace=True) # dataframe index 재설정
df

,title,url,date
0,"애플, 14년 만에 자체 칩 탑재 '맥북' 출시하나…10일 공개",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
1,“1004 주인 찾습니다!” 400대1 골드번호 도전! [IT선빵!],https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
2,"KT, 국내 최초 `5G 통합 코어망` 구축… SA 상용화 성큼",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
3,"SKT, 獨 5G AI 빅데이터 기술 언택트 전수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
4,"네이버 개발자 컨퍼런스 `데뷰2020`, 이달 10일부터 참가 접수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03


### 또한 헤드라인 뉴스안에는 전날에 생성된 헤드라인 뉴스도 포함되어 있기 때문에
### 전날의 뉴스를 저장하고 오늘 기사와 비교하여 중복된 기사를 제거하였습니다

In [8]:
# 이전 뉴스 기사 제목 Load
save_news = pd.read_csv('C:/driver/news_save/save_news.csv',encoding='UTF-8')
save_news.tail()

,title,url,date
60,"KT, 국내 최초 ‘5G 통합 코어망’ 구축…SA 상용화 성큼",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
61,"KT, 국내 최초 '5G 통합 코어망' 구축",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
62,"KT, 국내 최초 5G 통합 코어망 구축…""5G 단독모드 상용화 성큼""",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
63,"네이버 개발자 컨퍼런스 `데뷰2020`, 이달 10일부터 참가 접수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
64,"네이버 개발자 컨퍼런스 DEVIEW 2020, 10일부터 접수..인원제한 없어",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03


In [9]:
today = datetime.today().strftime("%Y-%m-%d")
today = pd.to_datetime(today, format='%Y-%m-%d')
drop_list = []
prev_list = []
for row, prev_date in enumerate(save_news.date) :
    prev_date = pd.to_datetime(prev_date, format='%Y-%m-%d')
    days= str(today-prev_date)
    days = int(days[:1])
    if days >= 5 :
        drop_list.append(row)
        continue

save_news = save_news.drop(drop_list).reset_index(drop=True)
        
prev_title = save_news.title.tolist()        
        

duplicate_title(prev_title,df.title,prev_list)
        
df.drop(prev_list).reset_index(drop=True,inplace=True)

In [10]:
# # 데이터 저장
save_news = pd.concat([save_news,raw_df]) # 이전 뉴스에 오늘자 뉴스 추가
save_news.drop_duplicates(subset=['title'], keep='first',inplace=True)
save_news.to_csv('C:/driver/news_save/save_news.csv',encoding='UTF-8',index=False) # 추가한 데이터 저장

#### - 네이버 뉴스는 IT/과학이기 때문에 과학과 관련된 단어 리스트 추가를 통해
#### - 과학과 관련된 뉴스를 제거하였습니다

In [11]:
df

,title,url,date
0,"애플, 14년 만에 자체 칩 탑재 '맥북' 출시하나…10일 공개",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
1,“1004 주인 찾습니다!” 400대1 골드번호 도전! [IT선빵!],https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
2,"KT, 국내 최초 `5G 통합 코어망` 구축… SA 상용화 성큼",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
3,"SKT, 獨 5G AI 빅데이터 기술 언택트 전수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
4,"네이버 개발자 컨퍼런스 `데뷰2020`, 이달 10일부터 참가 접수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03


In [12]:
science_list = ['노벨', '화학','과학자','후쿠시마','세포','방사능','방사선','세포','암','지렁이','독성','플라스틱',
                '바이러스','미생물','가위', '세포의','코로나','코로나19','초미세','가위로','감염',
                '유체','개체가','초미세먼지','유전자','유전자가위로','치료제', '소행성' ,'태양계' ,'탐사선', '토양',
                'DNA','오염수','국감','세슘','원전','미세먼지'] # 1과학 단어 list

serial_list = ['이슈IN','슬기로운 소비생활','사이언스샷','핵잼 사이언스','사이언스카페','이덕환의 과학세상',
              '사이언스','강석기의과학카페','넘버스','콘텐츠뷰', '표지로 읽는 과학','고든 정의 TECH+','김우섭 기자의 바이오 탐구영역']

In [13]:
science_drop = []
for row, science in enumerate(df.title) :
    count=0
    for i in serial_list:
        p = re.compile(r"\[(.+)\](.+)")
        m= p.match(science)
        if science.count(i)>=1 :
            science_drop.append(row)
            break
        
    title_split = science.replace("]", "] ")
    title_split = re.sub("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣| a-zA-Z0-9]", " ",title_split) # 특수문자 제거
    title_split = re.split(' ', title_split) # 기사 제목을 띄어쓰기 단위로 구분
    title_split = [ string for string in title_split if string != ''] #  리스트에 빈공간인 ''로 저장되는 list 삭제

    
    for k in science_list :
        if title_split.count(k)>=1 :
            count +=1
            if count>=2 :
                count=0 
                science_drop.append(row)
                break
science_drop
df.drop(science_drop,inplace=True)

In [14]:
df

,title,url,date
0,"애플, 14년 만에 자체 칩 탑재 '맥북' 출시하나…10일 공개",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
1,“1004 주인 찾습니다!” 400대1 골드번호 도전! [IT선빵!],https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
2,"KT, 국내 최초 `5G 통합 코어망` 구축… SA 상용화 성큼",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
3,"SKT, 獨 5G AI 빅데이터 기술 언택트 전수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03
4,"네이버 개발자 컨퍼런스 `데뷰2020`, 이달 10일부터 참가 접수",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-11-03


### 크롤링한 기사 URL에 접속하여 제목과 본문을 추출하여 저장하였습니다.

In [15]:
news_url =df['url'].tolist()  ## 각 뉴스 URL 주소 입력
final_article = [] # 뉴스 본문 저장 LIST 생성

for page in news_url :
    
    res = requests.get(page, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) ## 뉴스 URL 주소 접속
    soup = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = soup.select("#articleBodyContents") # 기사 본문 저장되어 있는 html주소 복사


    news_content = content[0] # select된 html중 기사는 0번째 리스트에 존재하며 이를 저장


    for target in news_content.find_all('script'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('a'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('iframe'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()


    for target in news_content.find_all('strong'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()


    article_list = str(news_content).replace('&amp;apos;','').replace('\n','').replace('\t','').replace('▶','').replace('img_desc">','img_desc"> <br/><br/>')
    # 전처리한 네이버 기사 저장
    final_article.append(article_list)
    # 3 곳의 기사를 통합하기 위해 통합 List에 저장
    time.sleep(10)

final_title = df.title.tolist()

## ITWorld Korea 머신러닝, 클라우드, 빅데이터 태그 크롤링

> ITWorld 날짜의 경우 기사 게시 일주일 이전에는 n분 전, n 시간 전, n일 전, 일주일 후에는 yyyy.mm.dd로 변경되기 때문에  
> 4가지 경우에 따라 데이터를 추출하는 작업이 필요합니다   
>  
> 해당 자료를 모두 정제하여 파이썬 표준 datetime인 yyyy-mm-dd로 변경하였습니다
> 

In [16]:
itw_news = ['http://www.itworld.co.kr/t/54649/%EB%B9%85%20%EB%8D%B0%EC%9D%B4%ED%84%B0',
               'http://www.itworld.co.kr/t/69500/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D',
               'http://www.itworld.co.kr/t/34/%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C']
               ## ITWorld 머신러닝, 클라우드, 빅데이터 tag URL주소
title_list =[] # 제목을 저장하기 위한 list 
url_list = [] # URL을 저장하기 위한 list 
total_date = [] # 닐짜를 저장하기 위한 list 
tag_list = [] # tag를 저장하기 위한 list

it_date_class = 'news_list_time default_small_font color_lightest_gray'
    
for itw in itw_news :
    res = requests.get(itw, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
    # requests이용하여 뉴스 주소 Load, 네이버 크롬 차단 방지 위해 chrome version을 입력하고 뉴스 주소를 설정
    # SSL ERROR 방지를 위해 Verify False 설정
    soup = BeautifulSoup(res.text, "html.parser") # html기준으로 parsing


    # 제목, 링크 출력 ---------------------------(2)
    it_title = soup.select('div > h4 > a') # 제목 링크가 존재하는 tag
    for i in it_title :
        title_list.append(str.strip(i.string)) #기록된 기사 제목을 리스트에 추가
        itworld_url = 'http://www.itworld.co.kr'+ i.attrs['href'] # 기사 url 추출
        url_list.append(itworld_url) # 기사 url 저장


    # # 날짜 추출
    itworld_cio_date(soup,it_date_class)
    
    ## tag 추출
    itw_tag = soup.find_all("div",{"class":"color_lightest_gray"}) # 각 기사 타입을 추출하는 코드

    for tag in itw_tag :
        tag_seg = re.findall(r'\d+', tag.get_text())
        if not tag_seg:
            tag = tag.get_text()
            tag = tag.replace('\n','').replace('\t','').replace(' ','')
            tag_list.append(tag)



### ITWorld의 머신러닝, 클라우드, 빅데이터 중 두 곳 이상에 모두 게시된  중복 기사가 존재합니다  
### 이를 위해 세 군데에서 수집한 기사를 모두 한 곳으로 모아 중복을 제거하였습니다
### 또한 네이버 기사와 마찬가지로 전날과 중복 방지를 위해 이전에 저장한 기사를 통해 중복을 제거하였습니다

In [17]:
itw_df = pd.DataFrame({'title': title_list,'url':url_list,'date':total_date,'tag':tag_list}) 
# 기사제목과, URL을 dataframe에 임시 저장
df_common_prep(itw_df)
itw_df['date'] = itw_df['date'].str[:11].str.strip() # 날짜의 앞뒤 공백 제거
itw_df.head()

,title,url,date,tag
0,데이터 기술 투자 ROI를 높이는 3가지 방법,http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-11-03,빅데이터
1,"맵퍼스-도로공사, “사고포착알리미 시행 후 돌발상황 인지 시간 단축”",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-27,BI/분석/빅데이터/애플리케이션
2,"더존비즈온, 3분기 매출 730억 원 발표…“지난해 비해 20.3% 증가”",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-27,빅데이터/애플리케이션
3,"SKT-파이어김, 화재 징후 데이터 분석 기반 자동소화 시스템 구축 사업 위한 MO...",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-26,빅데이터/사물인터넷
4,"현대BS&C, 세종공업 스마트공장 시범공장 ‘용접 모니터링 시스템’ 구축 완료",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-26,머신러닝/빅데이터/사물인터넷/애플리케이션


In [18]:
# itworld prev data load
itw_prev = pd.read_csv('C:/driver/news_save/itw.csv',encoding='UTF-8') # ITWORLD 이전 기사 LOAD

In [19]:
itw_prev_title = itw_prev.title.tolist() # 이전 기사 제목 list 생성
itw_prev_list = [] # 중복 기사 판단 list
duplicate_title(itw_prev_title,itw_df.title,itw_prev_list) # 이전 날짜와 중복된 기사 제거

        
itw_df.drop(itw_prev_list,inplace=True)
itw_df

,title,url,date,tag


In [20]:
itw_prev = pd.concat([itw_prev,itw_df]) # 오늘 날짜의 기사를 이전 날짜에 편입
itw_prev.to_csv('C:/driver/news_save/itw.csv',encoding='UTF-8',index=False) # 저장


In [21]:
itw_url =itw_df['url'].tolist()  ## 각 뉴스 URL 주소 입력

for page in itw_url :
    
    res = requests.get(page, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) ## 뉴스 URL 주소 접속
    news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = news_html.select(".node_body") # 기사 본문 저장되어 있는 html주소 복사


    news_content = content[0]


    article_list = str(news_content).replace('src="','src="http://www.itworld.co.kr').replace('&amp;apos;','').replace('\n','').replace('\t','')
    # 추출한 기사 전처리 
    article_list = article_list.replace('ⓒ', '').replace(u'\xa0',u'').replace('\u200b','').replace('\r','')
    # 추출한 기사 전처리 2
    final_article.append(article_list) # 전처리한 기사를 naver 뉴스 저장한 통합 list에 저장
    time.sleep(10)

final_title.extend(itw_df.title.tolist()) # 기사 제목을 naver 기사 저장한 통합 list에 저장

### ZDNet 인공지능 태그 크롤링

ZDNet의 경우 원격 접속이 막혀있어 현재 노트북의 크롬 user agent를 설정하여 접속하였습니다.  
향후 다른 사람의 컴퓨터로 사용하거나 크롬 버전이 변경될 경우 headers parameter에 'User-Agent'를 변경하여 사용해야 합니다.  



In [22]:
page = 'http://zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
# zdnet 인공지능 url
res = requests.get(page, headers={'User-Agent':
                                  ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                   "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                   verify = False)

news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
content = news_html.select(".news_box") # 기사 본문 저장되어 있는 html주소 복사

zd_title_list =[] # 제목을 저장하기 위한 list
zd_url_list = [] # URL을 저장하기 위한 list
zd_date_list = [] # 날짜를 저장하기 위한 list
zdcontent = content[0].select('div > a') # 기사가 저장되어 있는 링크 추출

# 제목, 링크 출력 ---------------------------(2)
for result in zdcontent:
    if result.h3 == None: # 값이 존재하지 않을 경우 해당 루프 건너 뜀
        continue
    zd_title_list.append(str.strip(result.h3.string)) #기록된 기사 제목을 리스트에 추가
    zd_url_list.append('https://zdnet.co.kr' + result.attrs['href']) #기록된 기사 URL을 리스트에 추가
    
zd_date = content[0].select('.byline') # 날짜를 추출하기 위한 class select

# zdnet의 날짜는 yyyy.dd.mm 이후 AM, PM 이 기록되고, 이를 기준으로 다른 문자열이 나온다. 
# 이를 해결하기 위해 AM, PM기준으로 문자열을 자르고 앞의 문자열인 날짜를 남기고 전처리를 진행한다.

for date in zd_date : 
    date = date.span.string
    if 'AM' in date:
        date = str.strip(date.split("AM")[0]) # AM기준으로 split
    
    else : date = str.strip(date.split("PM")[0]) # PM기준으로 split
    date = date.replace('.','-')
    zd_date_list.append(date)

zd_df = pd.DataFrame({'title': zd_title_list,'url':zd_url_list,'date':zd_date_list}) # 기사제목과, URL을 dataframe에 임시 저장
df_common_prep(zd_df)
zd_df.head()

,title,url,date
0,"네이버, '인터스피치 2020'에 클로바 중심 논문 10개 채택 성과",https://zdnet.co.kr/view/?no=20201102134136,2020-11-02
1,"""팬데믹 일상 속 AI 기술 방향은""…'삼성 AI 포럼' 개최",https://zdnet.co.kr/view/?no=20201102101552,2020-11-02
2,"에이치비스미스, 아마존 '정글' 지원에 선정",https://zdnet.co.kr/view/?no=20201102100631,2020-11-02
3,"KAIST, 차세대 AI 국제워크숍 5일 개최",https://zdnet.co.kr/view/?no=20201101105709,2020-11-02
4,"문동준 씨에스피아이 이사 ""AI로 바닷속 깨끗이 할 터""",https://zdnet.co.kr/view/?no=20201102093449,2020-11-02


In [23]:
# zdnet prev data load
zd_prev = pd.read_csv('C:/driver/news_save/zd.csv',encoding='UTF-8')

In [24]:
zd_prev_title = zd_prev.title.tolist() # zdnet의 기사 제목 저장
zd_prev_list = []
duplicate_title(zd_prev_title,zd_df.title,zd_prev_list)

        
zd_df.drop(zd_prev_list,inplace=True) #기사 중복 제거

In [25]:
zd_prev = pd.concat([zd_prev,zd_df]) # 이전 기사에 오늘 날짜 기사 추가
zd_prev.to_csv('C:/driver/news_save/zd.csv',encoding='UTF-8',index=False) #저장

In [26]:
zd_df.head()

,title,url,date


#### ZDNet의 인공지능 태그는 매일 글이 올라오지 않는 경우가 많아 당일 기사가 없을 경우에는 SKIP합니다

In [27]:
zd_url =zd_df['url'].tolist()  ## 각 뉴스 URL 주소 입력


for page in zd_url :
    if not page:
        break

    res = requests.get(page, headers={'User-Agent':
                                      ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                       "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                       verify = False) 
    ## 뉴스 URL 주소 접속
    news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = news_html.select("#content") # 기사 본문 저장되어 있는 html주소 복사


    zdcontent = content[0]


    for target in zdcontent.find_all('script'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in zdcontent.find_all('section'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()


    for target in zdcontent.find_all('h2'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('iframe'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    zdcontent = zdcontent.find_all(['p','figure']) # 기사 본문 추출

    zd_article ='' # 광고를 제거하고 기사 저장하기 위한 빈 문자열 생성
    for k in zdcontent :
        zd_article = zd_article + str(k) # 기사 에 해당 본문 추출한 tag 문자열 추가


    article_list = zd_article.replace('&amp;apos;',"").replace('\n','').replace('\t','').replace('\xa0','') #기사 본문 전처리

    final_article.append(article_list)
    time.sleep(10)
final_title.extend(zd_df.title.tolist())

### CIO Korea 빅데이터, AI, 클라우드

In [28]:
cio_news = ['http://www.ciokorea.com/t/2996/%EB%B9%85%20%EB%8D%B0%EC%9D%B4%ED%84%B0',
           'http://www.ciokorea.com/t/32/%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C',
           'http://www.ciokorea.com/t/22000/AI']
               ##  CIO Korea 빅데이터, AI, 클라우드 URL주소
title_list =[] # 제목을 저장하기 위한 list 
url_list = [] # URL을 저장하기 위한 list 
total_date = [] # 닐짜를 저장하기 위한 list 
tag_list = [] # tag를 저장하기 위한 list

cio_date_class = 'list_time default_small_font color_lightest_gray font_12 fr'
    
for page in cio_news :
    res = requests.get(page, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
    # requests이용하여 뉴스 주소 Load, 네이버 크롬 차단 방지 위해 chrome version을 입력하고 뉴스 주소를 설정
    # SSL ERROR 방지를 위해 Verify False 설정
    soup = BeautifulSoup(res.text, "html.parser") # html기준으로 parsing


    # 제목, 링크 출력 ---------------------------(2)
    cio_title = soup.select('div > h4 > a') # 제목 링크가 존재하는 tag
    for i in cio_title :
        title_list.append(str.strip(i.string)) #기록된 기사 제목을 리스트에 추가
        cio_url = 'http://www.ciokorea.com/'+ i.attrs['href'] # 기사 url 추출
        url_list.append(cio_url) # 기사 url 저장


    # # 날짜 추출

    itworld_cio_date(soup,cio_date_class)

In [29]:
cio_df = pd.DataFrame({'title': title_list,'url':url_list,'date':total_date}) 
# 기사제목과, URL을 dataframe에 임시 저장
df_common_prep(cio_df)
cio_df['date'] = cio_df['date'].str[:11].str.strip() # 날짜의 앞뒤 공백 제거
cio_df.head()

,title,url,date
0,김진철의 How-to-Big Data | 빅데이터의 미래 (12),http://www.ciokorea.com//t/2996/%EB%B9%85%20%E...,2020-10-31
1,"더존비즈온, 3분기 매출 730억 원 발표… “전년대비 20.3% 증가”",http://www.ciokorea.com//t/2996/%EB%B9%85%20%E...,2020-10-27
2,"SKT-파이어김, 화재 징후 데이터 분석 기반 자동소화 시스템 구축 사업 위한 MO...",http://www.ciokorea.com//t/2996/%EB%B9%85%20%E...,2020-10-26
3,"현대BS&C, 세종공업 스마트공장 시범공장 ‘용접 모니터링 시스템’ 구축 완료",http://www.ciokorea.com//t/2996/%EB%B9%85%20%E...,2020-10-26
4,유실·인종차별·개인정보 침해···애널리틱스 및 AI 결함이 빚은 참사 5건,http://www.ciokorea.com//t/2996/%EB%B9%85%20%E...,2020-10-26


In [32]:
# itworld prev data load
cio_prev = pd.read_csv('C:/driver/news_save/cio.csv',encoding='UTF-8') # cio 이전 기사 LOAD

In [33]:
cio_prev_title = cio_prev.title.tolist() # 이전 기사 제목 list 생성
cio_prev_list = [] # 중복 기사 판단 list
duplicate_title(cio_prev_title,cio_df.title,cio_prev_list) # 이전 날짜와 중복된 기사 제거

        
cio_df.drop(cio_prev_list,inplace=True)
cio_df

,title,url,date


In [34]:
cio_prev = pd.concat([cio_prev,cio_df]) # 오늘 날짜의 기사를 이전 날짜에 편입
cio_prev.to_csv('C:/driver/news_save/cio.csv',encoding='UTF-8',index=False) # 저장


In [79]:
cio_url =cio_df['url'].tolist()  ## 각 뉴스 URL 주소 입력

for page in cio_url :
    
    res = requests.get(page, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) ## 뉴스 URL 주소 접속
    news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = news_html.select(".node_body") # 기사 본문 저장되어 있는 html주소 복사


    news_content = content[0]


    article_list = str(news_content).replace('src="','src="http://www.ciokorea.com').replace('&amp;apos;','').replace('\n','').replace('\t','')
    # 추출한 기사 전처리 
    article_list = article_list.replace('ⓒ', '').replace(u'\xa0',u'').replace('\u200b','').replace('\r','').replace('\'','')
    # 추출한 기사 전처리 2
    final_article.append(article_list) # 전처리한 기사를 naver 뉴스 저장한 통합 list에 저장
    time.sleep(10)

final_title.extend(cio_df.title.tolist()) # 기사 제목을 naver 기사 저장한 통합 list에 저장

#### 사이트 세 곳에서 기사를 가져왔고 네이버와 중복될 가능성이 있기 때문에
#### 기사제목을 종합하여 중복을 제거하고 진행합니다.

In [35]:
final_df = pd.DataFrame({'title':final_title, 'article':final_article}) # 기사제목과, 기사를 dataframe에 임시 저장
final_df.drop_duplicates(subset=['title'], keep='first',inplace=True)
final_df.dropna(inplace=True)

In [36]:
final_df.head()

,title,article
0,"애플, 14년 만에 자체 칩 탑재 '맥북' 출시하나…10일 공개","<div class=""_article_body_contents"" id=""articl..."
1,“1004 주인 찾습니다!” 400대1 골드번호 도전! [IT선빵!],"<div class=""_article_body_contents"" id=""articl..."
2,"KT, 국내 최초 `5G 통합 코어망` 구축… SA 상용화 성큼","<div class=""_article_body_contents"" id=""articl..."
3,"SKT, 獨 5G AI 빅데이터 기술 언택트 전수","<div class=""_article_body_contents"" id=""articl..."
4,"네이버 개발자 컨퍼런스 `데뷰2020`, 이달 10일부터 참가 접수","<div class=""_article_body_contents"" id=""articl..."


In [37]:
final_df.to_csv('C:/driver/news_save/finaldf.csv',encoding='UTF-8',index=False)